<a href="https://colab.research.google.com/github/PranavBharadwaj-1328/Stock-Purchase-Recommender/blob/main/stockClosePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required packages
! pip install pyspark
! pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Processing

In [ ]:
# Initial imports
import os
import numpy as np
import glob
import datetime
import ipywidgets as widgets
import yfinance as yf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In [ ]:
# Initialize a spark session
spark = SparkSession \
        .builder \
        .master("local") \
        .appName("SRS") \
        .getOrCreate()

In [ ]:
class LR_model:
    '''A Linear Regressor which predicts closing price of stock'''
    def __init__(self):
        self.model = None

    def train(self, data):
        assembler = VectorAssembler(
            inputCols = ['Open', 'Low', 'High', 'SMA','CCI'],
            outputCol = 'featurevec',
            handleInvalid = "skip"
            )
        data = data.toPandas()
        data = spark.createDataFrame(data)
        data = assembler.transform(data)
        algo = LinearRegression(featuresCol='featurevec', labelCol = 'Close')
        self.model = algo.fit(data)

    def predict(self, data):
        assembler = VectorAssembler(
            inputCols = ['Open', 'Low', 'High', 'SMA', 'CCI'],
            outputCol = 'featurevec',
            handleInvalid = "skip"
            )
        data = assembler.transform(data)
        predictions = self.model.transform(data)
        return predictions


In [ ]:
class StockData:
    '''Fetch and add features to stock data of given tickers'''
    def __init__(self, tickers):
        self.tickers = tickers
        self.stock_data = self.fetch_data()
        self.models = {}
        self.add_features()

    def fetch_data(self):
        stock_data = {}
        data = yf.download(" ".join(self.tickers),
                           period="5y",
                           group_by='ticker'
                           )
        data = data.dropna()
        for ticker in self.tickers:
            stock_data[ticker] = spark.createDataFrame(
                data[ticker].reset_index()) \
                .withColumn("Ticker", lit(ticker)) \
                .withColumnRenamed('Adj Close', 'Adj_close')
        return stock_data

    def RSI(self, company_data):
        # Creating gain_loss column for each day, close_price_today - close_price_yesterday:
        gain_loss = company_data['Close'] - lag(company_data["Close"],1).over(Window().orderBy("Date"))
        # Calculating the Average of 14 days gains and losses
        gain = when(gain_loss > 0, gain_loss).otherwise(0)
        Avg_gain = avg(gain).over(Window.orderBy("Date").rowsBetween(-14, -1))
        loss = when(gain_loss < 0, gain_loss).otherwise(0)
        Avg_loss = avg(abs(loss)).over(Window.orderBy("Date").rowsBetween(-14, -1))
        # Calculating RSI:
        RS = Avg_gain / Avg_loss
        RSI = 100 - (100/(1+RS))
        return RSI

    def K(self, company_data):
        HH_14 = max('High').over(Window().orderBy("Date").rowsBetween(-14,0))
        LL_14 = min("Low").over(Window().orderBy("Date").rowsBetween(-14,0))
        K = 100*((company_data["Close"] - LL_14) / (HH_14 - LL_14))
        return K

    # Uptrend = 1, Downtrend = 0
    def add_features(self):
        i = 0
        w1 = Window.orderBy(col("Date")).rowsBetween(-1,0)
        w14 = Window.orderBy(col("Date")).rowsBetween(-14,-1)
        w141 = Window.orderBy(col("Date")).rowsBetween(-14,0)
        for ticker in self.tickers:
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("Today_trend", when(col("Close") > col("open"), 1).otherwise(0))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("Tommorow_trend", when(2*(col("Close") - avg("Close").over(w1)) > 0, 1).otherwise(0))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("SMA", avg("Close").over(w14))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("TP", (col("High") + col("Low") + col("Close"))/ 3)
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("14DTP", avg(col("TP")).over(w141))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("14MD", avg(abs(col("TP") - col("14DTP"))).over(w141))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("CCI", (col("TP") - col("14DTP"))/(0.15*col("14MD")))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("RSI", self.RSI(self.stock_data[ticker]))
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("K", self.K(self.stock_data[ticker]))
            self.stock_data[ticker] = self.stock_data[ticker].na.drop(subset=['SMA','RSI', 'K'])
            avgVolume = self.stock_data[ticker].agg({'Volume': 'avg'}).collect()[0][0]
            self.stock_data[ticker] = self.stock_data[ticker].withColumn("Normal_volume", col("Volume")/avgVolume)
            self.models[ticker] = LR_model()
            self.models[ticker].train(data=self.stock_data[ticker])

In [ ]:
ticker_widget = widgets.Textarea(
    placeholder = "Enter tickers for desired companies on individual lines",
    value = "MSFT\nAAPL\nGOOG\nIBM\nNKE\nAMZN\nNFLX\nTWTR",
    )
display(ticker_widget)

Textarea(value='MSFT\nAAPL\nGOOG\nIBM\nNKE\nAMZN\nNFLX\nTWTR', placeholder='Enter tickers for desired companie…

In [ ]:
tickers = ticker_widget.value.split("\n")
data = StockData(tickers)

[*********************100%***********************]  8 of 8 completed


# Stock price prediction

In [ ]:
# Importing necessary packages for LSTMs
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
stockData = {}
for k in data.stock_data:
    stockData[k] = data.stock_data[k].toPandas()

In [ ]:
def LSTM_model(ticker, col, look_back=20, period=30):
    '''Function which forecasts feature using LSTM'''
    def predict_dates(period):
        last_date = stockData[ticker]['Date'].values[-1]
        prediction_dates = pd.date_range(last_date, periods=period+1).tolist()
        return prediction_dates

    def predict(period, model):
        prediction_list = col_data[-look_back:]
        for _ in range(period):
            x = prediction_list[-look_back:]
            x = x.reshape((1, look_back, 1))
            out = model.predict(x)[0][0]
            prediction_list = np.append(prediction_list, out)
        prediction_list = prediction_list[look_back-1:]
        return prediction_list

    # Creating and training the model
    col_data = stockData[ticker][col].values
    col_data = col_data.reshape((-1,1))
    train_generator = TimeseriesGenerator(
        col_data,
        col_data,
        length=look_back,
        batch_size=20
        )
    model = Sequential()
    model.add(
        LSTM(
            10,
            activation='relu',
            input_shape=(look_back, 1)
            )
    )
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    num_epochs = 50
    model.fit_generator(train_generator, epochs=num_epochs, verbose=1)
    col_data = col_data.reshape((-1))

    # Forecasting future values
    forecast = predict(period, model)
    forecast_dates = predict_dates(period)
    return forecast_dates, forecast

In [ ]:
# Forecasting features for all companies
lstm_features = ["Open", "High", "Low", "SMA", "CCI"]
aug_data = {}
for ticker in tickers:
    for c in lstm_features:
        forecast_dates, forecast = LSTM_model(ticker, c, period=7, look_back=10)
        if ticker not in aug_data:
            aug_data[ticker] = {}
            aug_data[ticker]["Date"] = forecast_dates
        aug_data[ticker][c] = forecast

Epoch 1/50


<ipython-input-47-d6545430a111>:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


61/61 [==============================] - 2s 18ms/step - loss: 26029.4160
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 2765.4587
Epoch 3/50
61/61 [==============================] - 1s 18ms/step - loss: 23.8829
Epoch 4/50
61/61 [==============================] - 1s 17ms/step - loss: 26.5140
Epoch 5/50
61/61 [==============================] - 1s 19ms/step - loss: 20.7263
Epoch 6/50
61/61 [==============================] - 1s 17ms/step - loss: 17.8573
Epoch 7/50
61/61 [==============================] - 1s 18ms/step - loss: 18.1281
Epoch 8/50
61/61 [==============================] - 1s 18ms/step - loss: 17.6068
Epoch 9/50
61/61 [==============================] - 1s 18ms/step - loss: 17.2871
Epoch 10/50
61/61 [==============================] - 2s 26ms/step - loss: 17.5202
Epoch 11/50
61/61 [==============================] - 2s 33ms/step - loss: 17.0571
Epoch 12/50
61/61 [==============================] - 1s 19ms/step - loss: 17.8643
Epoch 13/50
61/61 [============

Epoch 1/50
61/61 [==============================] - 2s 18ms/step - loss: 153031.6406
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 18725.4023
Epoch 3/50
61/61 [==============================] - 1s 18ms/step - loss: 113.2080
Epoch 4/50
61/61 [==============================] - 1s 18ms/step - loss: 55.4464
Epoch 5/50
61/61 [==============================] - 1s 17ms/step - loss: 53.1182
Epoch 6/50
61/61 [==============================] - 1s 17ms/step - loss: 52.5008
Epoch 7/50
61/61 [==============================] - 1s 18ms/step - loss: 54.6508
Epoch 8/50
61/61 [==============================] - 1s 18ms/step - loss: 54.2291
Epoch 9/50
61/61 [==============================] - 1s 19ms/step - loss: 54.6366
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 52.0680
Epoch 11/50
61/61 [==============================] - 1s 19ms/step - loss: 54.0904
Epoch 12/50
61/61 [==============================] - 1s 19ms/step - loss: 53.4306
Epoch 13/50
61/61

Epoch 1/50
61/61 [==============================] - 2s 17ms/step - loss: 6312.8452
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 7941.2334
Epoch 3/50
61/61 [==============================] - 1s 18ms/step - loss: 87.6941
Epoch 4/50
61/61 [==============================] - 1s 18ms/step - loss: 74.0051
Epoch 5/50
61/61 [==============================] - 1s 17ms/step - loss: 65.6703
Epoch 6/50
61/61 [==============================] - 1s 17ms/step - loss: 59.0397
Epoch 7/50
61/61 [==============================] - 1s 17ms/step - loss: 55.1609
Epoch 8/50
61/61 [==============================] - 1s 17ms/step - loss: 51.9197
Epoch 9/50
61/61 [==============================] - 1s 17ms/step - loss: 49.5295
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 47.5590
Epoch 11/50
61/61 [==============================] - 1s 17ms/step - loss: 46.0745
Epoch 12/50
61/61 [==============================] - 1s 17ms/step - loss: 45.3037
Epoch 13/50
61/61 [==

...

Epoch 1/50
61/61 [==============================] - 2s 18ms/step - loss: 1850.7045
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 1824.6793
Epoch 3/50
61/61 [==============================] - 1s 19ms/step - loss: 280.6435
Epoch 4/50
61/61 [==============================] - 1s 19ms/step - loss: 6.3779
Epoch 5/50
61/61 [==============================] - 1s 19ms/step - loss: 4.8872
Epoch 6/50
61/61 [==============================] - 1s 19ms/step - loss: 4.7225
Epoch 7/50
61/61 [==============================] - 1s 19ms/step - loss: 4.7652
Epoch 8/50
61/61 [==============================] - 1s 19ms/step - loss: 4.7305
Epoch 9/50
61/61 [==============================] - 1s 19ms/step - loss: 4.6542
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 3.0912
Epoch 11/50
61/61 [==============================] - 1s 19ms/step - loss: 2.6350
Epoch 12/50
61/61 [==============================] - 1s 18ms/step - loss: 2.3828
Epoch 13/50
61/61 [==========

Epoch 1/50
61/61 [==============================] - 2s 18ms/step - loss: 1290.2966
Epoch 2/50
61/61 [==============================] - 1s 19ms/step - loss: 20.4447
Epoch 3/50
61/61 [==============================] - 1s 19ms/step - loss: 5.4221
Epoch 4/50
61/61 [==============================] - 1s 19ms/step - loss: 3.8013
Epoch 5/50
61/61 [==============================] - 1s 18ms/step - loss: 3.8797
Epoch 6/50
61/61 [==============================] - 1s 19ms/step - loss: 3.8040
Epoch 7/50
61/61 [==============================] - 1s 18ms/step - loss: 3.0806
Epoch 8/50
61/61 [==============================] - 1s 19ms/step - loss: 3.1443
Epoch 9/50
61/61 [==============================] - 1s 18ms/step - loss: 4.0612
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 2.9100
Epoch 11/50
61/61 [==============================] - 1s 19ms/step - loss: 2.9452
Epoch 12/50
61/61 [==============================] - 1s 20ms/step - loss: 3.1827
Epoch 13/50
61/61 [==============

Epoch 1/50
61/61 [==============================] - 2s 19ms/step - loss: 2137.9941
Epoch 2/50
61/61 [==============================] - 1s 19ms/step - loss: 1181.5588
Epoch 3/50
61/61 [==============================] - 1s 19ms/step - loss: 308.4460
Epoch 4/50
61/61 [==============================] - 1s 19ms/step - loss: 53.5687
Epoch 5/50
61/61 [==============================] - 1s 19ms/step - loss: 7.1266
Epoch 6/50
61/61 [==============================] - 1s 19ms/step - loss: 5.2858
Epoch 7/50
61/61 [==============================] - 1s 18ms/step - loss: 4.7105
Epoch 8/50
61/61 [==============================] - 1s 19ms/step - loss: 4.8059
Epoch 9/50
61/61 [==============================] - 1s 18ms/step - loss: 4.6711
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 4.5249
Epoch 11/50
61/61 [==============================] - 1s 18ms/step - loss: 3.8588
Epoch 12/50
61/61 [==============================] - 1s 19ms/step - loss: 4.0751
Epoch 13/50
61/61 [=========

Epoch 1/50
61/61 [==============================] - 2s 18ms/step - loss: 62.5011
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 4.7794
Epoch 3/50
61/61 [==============================] - 1s 18ms/step - loss: 2.1673
Epoch 4/50
61/61 [==============================] - 1s 18ms/step - loss: 0.9638
Epoch 5/50
61/61 [==============================] - 1s 18ms/step - loss: 0.3768
Epoch 6/50
61/61 [==============================] - 1s 18ms/step - loss: 0.2162
Epoch 7/50
61/61 [==============================] - 1s 17ms/step - loss: 0.1714
Epoch 8/50
61/61 [==============================] - 1s 18ms/step - loss: 0.1253
Epoch 9/50
61/61 [==============================] - 1s 17ms/step - loss: 0.2101
Epoch 10/50
61/61 [==============================] - 1s 18ms/step - loss: 0.2309
Epoch 11/50
61/61 [==============================] - 1s 18ms/step - loss: 0.1485
Epoch 12/50
61/61 [==============================] - 1s 18ms/step - loss: 0.1192
Epoch 13/50
61/61 [=================

Epoch 1/50
61/61 [==============================] - 2s 18ms/step - loss: 65.0649
Epoch 2/50
61/61 [==============================] - 1s 18ms/step - loss: 43.8184
Epoch 3/50
61/61 [==============================] - 1s 18ms/step - loss: 32.3283
Epoch 4/50
61/61 [==============================] - 1s 18ms/step - loss: 26.2374
Epoch 5/50
61/61 [==============================] - 1s 18ms/step - loss: 23.8193
Epoch 6/50
61/61 [==============================] - 1s 18ms/step - loss: 22.7290
Epoch 7/50
61/61 [==============================] - 1s 19ms/step - loss: 21.9960
Epoch 8/50
61/61 [==============================] - 1s 19ms/step - loss: 21.4654
Epoch 9/50
61/61 [==============================] - 1s 19ms/step - loss: 21.0259
Epoch 10/50
61/61 [==============================] - 1s 19ms/step - loss: 20.7926
Epoch 11/50
61/61 [==============================] - 1s 19ms/step - loss: 20.4640
Epoch 12/50
61/61 [==============================] - 1s 18ms/step - loss: 20.2237
Epoch 13/50
61/61 [======

In [ ]:
# Converting forecasted data into Spark DataFrame to pass to the
# linear regression model in Spark
forecastDf = {}
for ticker in aug_data:
    forecastDf[ticker] = pd.DataFrame(aug_data[ticker])
pred_data = {}
for ticker in forecastDf:
    pred_data[ticker] = spark.createDataFrame(
        forecastDf[ticker],
        schema=["Date", "Open", "High", "Low", "SMA", "CCI"]
        )

In [ ]:
# Getting predictions for closing price for all companies
predictions = {}
for ticker in pred_data:
    predictions[ticker] = data.models[ticker].predict(pred_data[ticker])

# Recommendation

In [ ]:
predicts = {}
for ticker in predictions:
    predictions[ticker] = predictions[ticker].select(
        'Date',
        'Open',
        'High',
        'Low',
        'SMA',
        'CCI',
        'featurevec',
        'prediction',
        lit(col('prediction') - data.stock_data[ticker].tail(1)[0]['Close']).alias('profit')
        )

In [ ]:
daywise = {}
for day in aug_data['MSFT']['Date']:
    temp = []
    for key in predictions:
        temp.append([
            key,
            predictions[key].select('profit').where(col('Date') == day).collect()[0]['profit']
            ])
    temp = sorted(temp, key= lambda x: x[1], reverse=True)
    daywise[str(day).split()[0]] = temp

In [ ]:
print('Date \t\t Preference list')
for day in daywise:
    print(day, [x[0] for x in daywise[day]])

Date 		 Preference list
2022-10-27 ['NFLX', 'MSFT', 'AAPL', 'NKE', 'IBM', 'GOOG', 'TWTR', 'AMZN']
2022-10-28 ['MSFT', 'GOOG', 'AAPL', 'AMZN', 'TWTR', 'NKE', 'IBM', 'NFLX']
2022-10-29 ['GOOG', 'MSFT', 'AAPL', 'AMZN', 'TWTR', 'NKE', 'IBM', 'NFLX']
2022-10-30 ['GOOG', 'AAPL', 'MSFT', 'AMZN', 'TWTR', 'NKE', 'NFLX', 'IBM']
2022-10-31 ['MSFT', 'GOOG', 'AAPL', 'NFLX', 'AMZN', 'TWTR', 'NKE', 'IBM']
2022-11-01 ['GOOG', 'NFLX', 'AAPL', 'MSFT', 'AMZN', 'TWTR', 'NKE', 'IBM']
2022-11-02 ['GOOG', 'NFLX', 'MSFT', 'AAPL', 'AMZN', 'TWTR', 'IBM', 'NKE']
2022-11-03 ['MSFT', 'GOOG', 'NFLX', 'AAPL', 'AMZN', 'TWTR', 'IBM', 'NKE']
